In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
import numpy as np

import os
import cProfile
from tqdm import tqdm
import gde.database_io.dims
import gde.database_io.faks

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from sqlalchemy import func
from sqlalchemy.orm import aliased
import plotly.express as px

[05/06/24 10:57:51] INFO     Custom team name replacements loaded from                                ]8;id=288464;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=317257;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py#85\85]8;;\
                             /home/morten/soccerdata/config/teamname_replacements.json.                            

                    INFO     Custom league dict loaded from                                          ]8;id=514276;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=945100;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/soccerdata/_config.py#163\163]8;;\
                             /home/morten/soccerdata/config/league_dict.json.                                      

In [3]:
db_path = "/home/morten/Develop/packing-report/gde/GDE.db"

engine = create_engine(f'sqlite:///{db_path}', echo=False)
session = Session(engine)


# Joining the subqueries
query = (
    session.query(
        gde.database_io.dims.Games.expected_game_result,
        # gde.database_io.dims.Games.result
    ).filter(gde.database_io.dims.Games.minutes > 80)
    .filter(gde.database_io.dims.Games.expected_game_result >= 0)
    .filter(gde.database_io.dims.Games.version == 0.2)
)

results = query.all()

In [4]:
np.mean(results)

0.9862726799836052

In [5]:
px.histogram(pd.DataFrame(results, columns=['expected_game_result']))

Get Data. Train + Test

In [82]:
db_path = "/home/morten/Develop/packing-report/gde/GDE.db"

engine = create_engine(f'sqlite:///{db_path}', echo=False)
session = Session(engine)


# Subquery for result_table
ranked_subquery = (
    session.query(
        gde.database_io.dims.Games.game_id,
        gde.database_io.dims.Games.team_id,
        gde.database_io.dims.Games.result,
        func.row_number().over(
            partition_by=(gde.database_io.dims.Games.game_id, gde.database_io.dims.Games.team_id),
            order_by=func.count().desc()
        ).label('rank')
    )
    .filter(gde.database_io.dims.Games.game_date > '2018-06-01')
    .group_by(gde.database_io.dims.Games.game_id, gde.database_io.dims.Games.team_id, gde.database_io.dims.Games.result)
    .subquery()
)

result_table = (
    session.query(ranked_subquery.c.game_id, ranked_subquery.c.team_id, ranked_subquery.c.result)
    .filter(ranked_subquery.c.rank == 1)
    .subquery()
)

# Subquery for elo_table
elo_table = (
    session.query(
        func.avg(gde.database_io.dims.Games.elo).label('avg_elo'),
        gde.database_io.dims.Games.team_id,
        gde.database_io.dims.Games.game_id,
        gde.database_io.dims.Games.game_date,
        gde.database_io.dims.Games.home
    )
    .filter(gde.database_io.dims.Games.game_date > '2018-06-01')
    .group_by(gde.database_io.dims.Games.game_id, gde.database_io.dims.Games.team_id)
    .subquery()
)

# Joining the subqueries
query = (
    session.query(
        result_table.c.game_id,
        result_table.c.team_id,
        result_table.c.result,
        elo_table.c.avg_elo,
        elo_table.c.game_date, 
        elo_table.c.home

    )
    .join(elo_table, 
          (result_table.c.game_id == elo_table.c.game_id) & 
          (result_table.c.team_id == elo_table.c.team_id))
)

results = query.all()

In [83]:
data = pd.DataFrame(np.array(results), columns=["game_id", "team_id", "result", "avg_elo", "date", "home"])

In [84]:
data

,game_id,team_id,result,avg_elo,date,home
0,1284741,13,0-2,1697.619891075093,2018-08-12,1
1,1284741,167,2-0,1799.2587862928144,2018-08-12,0
2,1284742,183,2-0,1600.9694028398924,2018-08-11,1
3,1284742,188,0-2,1645.3972462913678,2018-08-11,0
4,1284743,162,2-0,1652.9139935200542,2018-08-11,0
...,...,...,...,...,...,...
7931,1557933,1143,0-2,1402.8862808037213,2022-05-15,0
7932,1557934,813,1-0,1365.8641117169866,2022-05-15,0
7933,1557934,1731,0-1,1404.44873320997,2022-05-15,1
7934,1557935,38,3-2,1527.0112867223588,2022-05-15,0


In [85]:
train_data = data[data["date"] <= "2021-07-01"].copy()
test_data = data[data["date"] > "2021-07-01"].copy()

In [86]:
train_data = pd.merge(train_data, train_data, how="outer", on="game_id")
train_data = train_data.loc[train_data["team_id_x"] != train_data["team_id_y"]]

test_data = pd.merge(test_data, test_data, how="outer", on="game_id")
test_data = test_data.loc[test_data["team_id_x"] != test_data["team_id_y"]]

In [87]:
train_data = train_data[train_data["home_x"] == "1"]
test_data = test_data[test_data["home_x"] == "1"]

MOV Regressor analysis

Create Model

In [89]:
train_data["goals_x"] = train_data["result_x"].apply(lambda x: int(x.split("-")[0]))
train_data["goals_y"] = train_data["result_y"].apply(lambda x: int(x.split("-")[0]))

In [90]:
train_data[train_data.columns] = train_data[train_data.columns].apply(pd.to_numeric, errors='ignore')

/tmp/ipykernel_92206/85079529.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  train_data[train_data.columns] = train_data[train_data.columns].apply(pd.to_numeric, errors='ignore')


In [91]:
train_data

,game_id,team_id_x,result_x,avg_elo_x,date_x,home_x,team_id_y,result_y,avg_elo_y,date_y,home_y,goals_x,goals_y
1,1284741,13,0-2,1697.619891,2018-08-12,1,167,2-0,1799.258786,2018-08-12,0,0,2
5,1284742,183,2-0,1600.969403,2018-08-11,1,188,0-2,1645.397246,2018-08-11,0,2,0
10,1284743,170,0-2,1643.147318,2018-08-11,1,162,2-0,1652.913994,2018-08-11,0,0,2
14,1284744,166,0-3,1602.843344,2018-08-11,1,15,3-0,1688.564055,2018-08-11,0,0,3
17,1284745,26,4-0,1721.353482,2018-08-12,1,29,0-4,1626.644871,2018-08-12,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11885,1485559,26,2-0,1708.945258,2021-05-23,1,162,0-2,1560.013634,2021-05-23,0,2,0
11890,1485560,167,5-0,1790.385411,2021-05-23,1,31,0-5,1626.657078,2021-05-23,0,5,0
11893,1485561,163,1-0,1496.235948,2021-05-23,1,184,0-1,1571.212447,2021-05-23,0,1,0
11898,1485562,29,3-0,1684.984590,2021-05-23,1,18,0-3,1591.344745,2021-05-23,0,3,0


In [101]:
import pymc as pm

with pm.Model() as game_model:
    home_advantage = pm.Normal("home_advantage", 0, 10)
    home_elo_diff = pm.Normal("home_elo", 0, 10) 
    away_elo_diff = pm.Normal("away_elo", 0, 10)

    # theta_h = home_advantage + home_elo_diff * (train_data["avg_elo_x"] - train_data["avg_elo_y"])
    theta_h = home_advantage + home_elo_diff * (train_data["avg_elo_x"] - train_data["avg_elo_y"])
    theta_a = away_elo_diff * (train_data["avg_elo_y"] - train_data["avg_elo_x"])

    goals_h = pm.Poisson("goals_h", pm.math.exp(theta_h), observed=train_data["goals_x"])
    goals_a = pm.Poisson("goals_a", pm.math.exp(theta_a), observed=train_data["goals_y"])

    # # observed
    # goal_diff = goals_h - goals_a

In [102]:
with game_model:
    trace = pm.sample(tune=2000)

[05/03/24 15:03:26] INFO     Auto-assigning NUTS sampler...                                             ]8;id=625283;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/pymc/sampling/mcmc.py\mcmc.py]8;;\:]8;id=221021;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/pymc/sampling/mcmc.py#713\713]8;;\

                    INFO     Initializing NUTS using jitter+adapt_diag...                              ]8;id=218746;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/pymc/sampling/mcmc.py\mcmc.py]8;;\:]8;id=959516;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/pymc/sampling/mcmc.py#1366\1366]8;;\

[05/03/24 15:03:34] INFO     Multiprocess sampling (4 chains in 4 jobs)                                 ]8;id=80929;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/pymc/sampling/mcmc.py\mcmc.py]8;;\:]8;id=318079;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/pymc/sampling/mcmc.py#805\805]8;;\

                    INFO     NUTS:                                                                      ]8;id=433421;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/pymc/sampling/mcmc.py\mcmc.py]8;;\:]8;id=951222;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/pymc/sampling/mcmc.py#249\249]8;;\

Output()

[05/03/24 15:06:23] INFO     Sampling 4 chains for 2_000 tune and 1_000 draw iterations (8_000 + 4_000  ]8;id=331397;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/pymc/sampling/mcmc.py\mcmc.py]8;;\:]8;id=172253;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/pymc/sampling/mcmc.py#891\891]8;;\
                             draws total) took 169 seconds.                                                        

Predict Test Data

In [103]:
test_data

,game_id,team_id_x,result_x,avg_elo_x,date_x,home_x,team_id_y,result_y,avg_elo_y,date_y,home_y,win_x,draw,win_y,sum
2,1549539,189,2-0,1599.753125,2021-08-13,1,13,0-2,1679.362119,2021-08-13,0,0.213,0.315,0.520,1.048
5,1549540,184,1-2,1556.526390,2021-08-14,1,211,2-1,1598.093822,2021-08-14,0,0.257,0.332,0.432,1.021
9,1549541,15,3-0,1705.047915,2021-08-14,1,162,0-3,1549.863180,2021-08-14,0,0.690,0.220,0.112,1.022
14,1549542,31,3-1,1614.017938,2021-08-14,1,18,1-3,1578.811778,2021-08-14,0,0.422,0.311,0.258,0.991
17,1549543,14,1-0,1633.876798,2021-08-14,1,161,0-1,1567.833038,2021-08-14,0,0.499,0.259,0.229,0.987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3949,1557931,1143,0-2,1406.130556,2022-05-07,1,4852,2-0,1440.326321,2022-05-07,0,0.258,0.325,0.406,0.989
3954,1557932,3429,0-0,1345.551615,2022-05-07,1,48,0-0,1439.409563,2022-05-07,0,0.160,0.271,0.517,0.948
3957,1557933,42,2-0,1539.170299,2022-05-15,1,1143,0-2,1402.886281,2022-05-15,0,0.624,0.197,0.115,0.936
3962,1557934,1731,0-1,1404.448733,2022-05-15,1,813,1-0,1365.864112,2022-05-15,0,0.458,0.280,0.256,0.994


In [104]:
test_data[test_data.columns] = test_data[test_data.columns].apply(pd.to_numeric, errors='ignore')

/tmp/ipykernel_92206/3560326254.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  test_data[test_data.columns] = test_data[test_data.columns].apply(pd.to_numeric, errors='ignore')


In [106]:
test_data["win_x"] = test_data.apply(lambda x: np.count_nonzero(np.random.poisson(np.exp(trace.posterior.home_advantage[0] + trace.posterior.home_elo[0] * (x["avg_elo_x"] - x["avg_elo_y"]))) - 
                                               np.random.poisson(np.exp(trace.posterior.away_elo[0] * (x["avg_elo_y"] - x["avg_elo_x"]))) > 0) / 1000, 
                                               axis=1)
test_data["draw"] = test_data.apply(lambda x: np.count_nonzero(np.random.poisson(np.exp(trace.posterior.home_advantage[0] + trace.posterior.home_elo[0] * (x["avg_elo_x"] - x["avg_elo_y"]))) - 
                                               np.random.poisson(np.exp(trace.posterior.away_elo[0] * (x["avg_elo_y"] - x["avg_elo_x"]))) == 0) / 1000, 
                                               axis=1)
test_data["win_y"] = test_data.apply(lambda x: np.count_nonzero(np.random.poisson(np.exp(trace.posterior.home_advantage[0] + trace.posterior.home_elo[0] * (x["avg_elo_x"] - x["avg_elo_y"]))) - 
                                               np.random.poisson(np.exp(trace.posterior.away_elo[0] * (x["avg_elo_y"] - x["avg_elo_x"]))) < 0) / 1000, 
                                               axis=1)

In [107]:
test_data["sum"] = test_data["win_x"] + test_data["draw"] + test_data["win_y"] # sanity check

In [108]:
test_data

,game_id,team_id_x,result_x,avg_elo_x,date_x,home_x,team_id_y,result_y,avg_elo_y,date_y,home_y,win_x,draw,win_y,sum
2,1549539,189,2-0,1599.753125,2021-08-13,1,13,0-2,1679.362119,2021-08-13,0,0.322,0.250,0.418,0.990
5,1549540,184,1-2,1556.526390,2021-08-14,1,211,2-1,1598.093822,2021-08-14,0,0.424,0.254,0.307,0.985
9,1549541,15,3-0,1705.047915,2021-08-14,1,162,0-3,1549.863180,2021-08-14,0,0.766,0.175,0.074,1.015
14,1549542,31,3-1,1614.017938,2021-08-14,1,18,1-3,1578.811778,2021-08-14,0,0.544,0.263,0.195,1.002
17,1549543,14,1-0,1633.876798,2021-08-14,1,161,0-1,1567.833038,2021-08-14,0,0.598,0.225,0.175,0.998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3949,1557931,1143,0-2,1406.130556,2022-05-07,1,4852,2-0,1440.326321,2022-05-07,0,0.399,0.266,0.317,0.982
3954,1557932,3429,0-0,1345.551615,2022-05-07,1,48,0-0,1439.409563,2022-05-07,0,0.284,0.275,0.405,0.964
3957,1557933,42,2-0,1539.170299,2022-05-15,1,1143,0-2,1402.886281,2022-05-15,0,0.718,0.181,0.106,1.005
3962,1557934,1731,0-1,1404.448733,2022-05-15,1,813,1-0,1365.864112,2022-05-15,0,0.541,0.252,0.190,0.983


In [109]:
def brier_multi(targets, probs):
    return np.mean(np.sum((probs - targets)**2, axis=1))

labels = [[1, 0, 0] if int(x.split("-")[0]) > int(x.split("-")[1]) else [0, 1, 0] if int(x.split("-")[0]) == int(x.split("-")[1]) else [0, 0, 1] for x in test_data.result_x]
probs = [[x, y, z] for x, y, z in zip(test_data.win_x, test_data.draw, test_data.win_y)]

In [110]:
brier_multi(np.array(labels), np.array(probs))

0.612019023185484